In [ ]:
# data_clean_book_sample.ipynb

import pandas as pd
from pathlib import Path

BASE = Path("../../raw_data/books")
books_path = BASE / "books.csv"
book_tags_path = BASE / "book_tags.csv"
tags_path = BASE / "tags.csv"

print("📘 Loading data...")
books = pd.read_csv(books_path)
book_tags = pd.read_csv(book_tags_path)
tags = pd.read_csv(tags_path)

print(f"books.csv: {books.shape}, book_tags.csv: {book_tags.shape}, tags.csv: {tags.shape}")


print("\n🔍 Columns:")
print("books.csv:", books.columns.tolist())
print("book_tags.csv:", book_tags.columns.tolist())
print("tags.csv:", tags.columns.tolist())


if "book_id" in book_tags.columns:
    book_tags = book_tags.rename(columns={"book_id": "goodreads_book_id"})


if "book_id" not in books.columns and "id" in books.columns:
    books = books.rename(columns={"id": "book_id"})


print("\n🔗 Merging book_tags and tags...")
book_tags = book_tags.merge(tags, on="tag_id", how="left")


book_tag_grouped = (
    book_tags.groupby("goodreads_book_id")["tag_name"]
    .apply(lambda x: ", ".join(x.head(10)))
    .reset_index()
    .rename(columns={"tag_name": "tag_list"})
)

print(f"✅ Created tag_list for {len(book_tag_grouped)} books")

print("\n🔗 Merging tags into books.csv...")
books = books.merge(book_tag_grouped, left_on="book_id", right_on="goodreads_book_id", how="left")


books["original_title"] = books["original_title"].fillna(books["title"])
books["authors"] = books["authors"].fillna("Unknown")
books["average_rating"] = pd.to_numeric(books["average_rating"], errors="coerce")
books["ratings_count"] = pd.to_numeric(books["ratings_count"], errors="coerce")
books["original_publication_year"] = pd.to_numeric(books["original_publication_year"], errors="coerce")

print(f"✅ Combined shape: {books.shape}")

print("\n🎯 Filtering high-rated, popular books...")
books_filt = books[
    (books["average_rating"] >= 4.0) &
    (books["ratings_count"] >= 5000) &
    (books["original_publication_year"].between(1900, 2025))
].copy()

print(f"📚 Remaining after filter: {len(books_filt)}")

C = books_filt["average_rating"].mean()
m = books_filt["ratings_count"].quantile(0.75)
v = books_filt["ratings_count"]
R = books_filt["average_rating"]
books_filt["weighted_score"] = (v/(v+m))*R + (m/(v+m))*C

books_filt = books_filt.sort_values("weighted_score", ascending=False)


book_sample = books_filt.head(100).copy()

out_path = Path("../../data_clean/book/book_sample_100.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)

keep_cols = [
    "book_id","title","original_title","authors","original_publication_year",
    "average_rating","ratings_count","tag_list","image_url","small_image_url"
]
keep_cols = [c for c in keep_cols if c in book_sample.columns]
book_sample[keep_cols].to_csv(out_path, index=False, encoding="utf-8")

print(f"\n✅ Saved {len(book_sample)} books → {out_path}")
print("sample:")
print(book_sample[["title","authors","average_rating","tag_list"]].head(5))


📘 Loading data...
books.csv: (10000, 23), book_tags.csv: (999912, 3), tags.csv: (34252, 2)

🔍 Columns:
books.csv: ['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year', 'original_title', 'title', 'language_code', 'average_rating', 'ratings_count', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url', 'small_image_url']
book_tags.csv: ['goodreads_book_id', 'tag_id', 'count']
tags.csv: ['tag_id', 'tag_name']

🔗 Merging book_tags and tags...
✅ Created tag_list for 10000 books

🔗 Merging tags into books.csv...
✅ Combined shape: (10000, 25)

🎯 Filtering high-rated, popular books...
📚 Remaining after filter: 5158

✅ Saved 100 books → ../../data_clean/book/book_sample_100.csv
示例:
                                                  title  \
421            Harry Potter Boxset (Harry Potter, #1-7)   
24    Harry Potter and the Deathly Hallows (Harry Po...   
1307 